In [1]:
import numpy as np
from scipy import ndimage
import os
import networkx as nx

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def generate_volume(shape=(10,10,10), fill_ratio=0.3, seed=42):
    """Generate a random 3D boolean volume where True represents occupied space."""
    np.random.seed(seed)
    return np.random.rand(*shape) < fill_ratio

def find_connected_components(volume, structure):
    """Label connected components in a 3D binary volume."""
    labeled_volume, num_components = ndimage.label(volume, structure=structure)  # Label each connected component
    return labeled_volume, num_components

def voxel_graph(component, label_value):
    """Create a graph from a single connected component."""
    G = nx.Graph()
    dims = component.shape
    for i in range(dims[0]):
        for j in range(dims[1]):
            for k in range(dims[2]):
                if component[i, j, k] == label_value:
                    G.add_node((i, j, k))
                    #for di, dj, dk in [(1,0,0), (-1,0,0), (0,1,0), (0,-1,0), (0,0,1), (0,0,-1)]: ## not the right connectivity ?
                    for di in range(-1,2,1):
                        for dj in range(-1,2,1):
                            for dk in range(-1,2,1):
                                ni, nj, nk = i + di, j + dj, k + dk
                                if 0 <= ni < dims[0] and 0 <= nj < dims[1] and 0 <= nk < dims[2] and not (ni==i and nj==j and nk==k): # and not (np.linalg.norm((ni-i, nj-j, nk-k))>1.5)
                                    if component[ni, nj, nk] == label_value:
                                        G.add_edge((i, j, k), (ni, nj, nk), weight=np.linalg.norm((ni-i, nj-j, nk-k)))
    return G

def find_minimax_center(G):
    """Find the node in the graph that minimizes the maximum geodesic distance to any other node."""
    lengths = dict(nx.all_pairs_dijkstra_path_length(G))  # Compute all-pairs shortest paths
    minimax_center = None
    min_max_dist = float('inf')

    for node in G.nodes():
        max_dist = max(lengths[node].values())  # Find the worst-case distance
        if max_dist < min_max_dist:
            min_max_dist = max_dist
            minimax_center = node
    
    return minimax_center, min_max_dist, lengths[minimax_center]

In [3]:
import anatomist.api as ana
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt
from soma import aims

a = ana.Anatomist()

global modules: /casa/host/build/share/anatomist-5.2/python_plugins
home   modules: /casa/home/.anatomist/python_plugins
loading module volumepalettes
loading module paletteViewer
loading module modelGraphs
loading module infowindow
loading module profilewindow
loading module bsa_proba
loading module meshsplit
loading module selection
loading module anacontrolmenu
loading module gltf_io
loading module ana_image_math
loading module palettecontrols
loading module statsplotwindow
loading module foldsplit
loading module histogram
loading module save_resampled
loading module simple_controls
loading module valuesplotwindow
loading module gradientpalette
all python modules loaded
Anatomist started.


# Visu components

In [79]:
#skel_dir = '/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/S.C.-sylv./mask/Lcrops'
skel_dir = '/neurospin/dico/data/deep_folding/current/datasets/UkBioBank40/crops/2mm/F.I.P./mask/Rcrops'
subjects_list = os.listdir(skel_dir)
subjects_list = [elem for elem in subjects_list if elem[-1]!='f']
subjects_list = subjects_list[:16]

volume_files = []
centroids_files = []
marked_volume_files = []
labeled_volume_files = []

for subject in subjects_list:
    skel_volume = aims.read(os.path.join(skel_dir, subject))
    skel_volume.np[skel_volume.np!=0]=1
    print(np.unique(skel_volume, return_counts=True))
    skel = skel_volume.np
    volume = skel[:,:,:,0]!=0
    structure = np.ones((3,3,3)) # we could also remove the edges
    #structure = None

    # Find connected components
    labeled_volume, num_components = find_connected_components(volume, structure=structure)
    print(f'Number of connected components : {num_components}')
    print(np.unique(labeled_volume, return_counts=True)[1])

    # Find minimax center for each connected component
    centers = {}
    for label_value in range(1, num_components + 1):  # Labels start from 1
        G = voxel_graph(labeled_volume, label_value)
        if G.number_of_nodes() > 0:
            center, max_dist, distances = find_minimax_center(G)
            centers[label_value] = (center, max_dist, distances)

    # dilate centroids
    centroids_im = np.zeros(volume.shape)
    for _, (center,_,_) in centers.items():
        centroids_im[center]=1

    #centroids_im = ndimage.binary_dilation(centroids_im, structure=structure)
    # create nifti and plot
    centroids_im = np.expand_dims(centroids_im, axis=-1).astype(np.float32)
    vol = aims.Volume(centroids_im)

    ## append centroids and skels in lists
    volume_files.append(skel_volume)
    centroids_files.append(vol)

    ## append marked skeleton
    skel_volume.np[vol.np!=0]=3 ## NB : MODIFIES THE ELEMENTS OF VOLUME FILES LIST ...
    print(np.unique(skel_volume, return_counts=True))
    marked_volume_files.append(skel_volume)

    # append labeled surfaces
    skel_volume.np[:]=np.expand_dims(labeled_volume, axis=-1)
    labeled_volume_files.append(skel_volume)

    # Print results
    #for label, (center, max_dist) in centers.items():
    #    print(f"Component {label}: Minimax center at {center}, max geodesic distance = {max_dist}")

(array([0, 1], dtype=int16), array([75335,  1885]))
Number of connected components : 17
[75335  1427     3     6     2     1   207     6    12    92     8    31
     9    66    10     3     1     1]
(array([0, 1], dtype=int16), array([75395,  1825]))
Number of connected components : 18
[75395    35  1480    85    48    24    10     1     1    15     3    19
     2     5    27    15    43     8     4]
(array([0, 1], dtype=int16), array([75241,  1979]))
Number of connected components : 18
[75241    15   585    16   719    10   185     1     1    15   248    88
     1    30    43     9     6     2     5]
(array([0, 1], dtype=int16), array([75146,  2074]))
Number of connected components : 14
[75146    85    10    52  1689     1    17    92     1    43     8    26
    47     2     1]
(array([0, 1], dtype=int16), array([75129,  2091]))
Number of connected components : 13
[75129    41  1762    59    53     5   124     3     1    14     5    12
    10     2]


KeyboardInterrupt: 

In [4]:
def build_gradient(pal):
    gw = ana.cpp.GradientWidget(None, 'gradientwidget', pal.header()['palette_gradients'])
    gw.setHasAlpha(True)
    nc = pal.shape[0]
    rgbp = gw.fillGradient(nc, True)
    rgb = rgbp.data()
    npal = pal.np['v']
    pb = np.frombuffer(rgb, dtype=np.uint8).reshape((nc, 4))
    npal[:, 0, 0, 0, :] = pb
    npal[:, 0, 0, 0, :3] = npal[:, 0, 0, 0, :3][:, ::-1]  # BGRA -> RGBA
    pal.update()

In [ ]:
block = a.createWindowsBlock(4)
dic_windows = {}

for i, vol in enumerate(marked_volume_files):
    dic_windows[f'a_vol{i}'] = a.toAObject(vol)
    pal = a.createPalette('sulci-model')
    pal.header()['palette_gradients'] = f"0;0;0;0;0.05;0.4667;0.05;0.4667;0.1;0.5333;0.1;0.5333;0.15;0;0.15;0;0.2;0;0.2;0;0.25;0;0.25;0;0.3;0;0.3;0;0.35;0;0.35;0;0.4;0;0.4;0;0.45;0;0.45;0;0.5;0;0.5;0;0.55;0;0.55;0;0.6;0;0.6;0;0.65;0.7333;0.65;0.7333;0.7;0.9333;0.7;0.9333;0.75;1;0.75;1;0.8;1;0.8;1;0.85;1;0.85;1;0.9;0.8667;0.9;0.8667;0.95;0.8;0.95;0.8;1;0.8;1;0.8#0;0;0;0;0.05;0;0.05;0;0.1;0;0.1;0;0.15;0;0.15;0;0.2;0;0.2;0;0.25;0.4667;0.25;0.4667;0.3;0.6;0.3;0.6;0.35;0.6667;0.35;0.6667;0.4;0.6667;0.4;0.6667;0.45;0.6;0.45;0.6;0.5;0.7333;0.5;0.7333;0.55;0.8667;0.55;0.8667;0.6;1;0.6;1;0.65;1;0.65;1;0.7;0.9333;0.7;0.9333;0.75;0.8;0.75;0.8;0.8;0.6;0.8;0.6;0.85;0;0.85;0;0.9;0;0.9;0;0.95;0;0.95;0;1;0.8;1;0.8#0;0;0;0;0.05;0.5333;0.05;0.5333;0.1;0.6;0.1;0.6;0.15;0.6667;0.15;0.6667;0.2;0.8667;0.2;0.8667;0.25;0.8667;0.25;0.8667;0.3;0.8667;0.3;0.8667;0.35;0.6667;0.35;0.6667;0.4;0.5333;0.4;0.5333;0.45;0;0.45;0;0.5;0;0.5;0;0.55;0;0.55;0;0.6;0;0.6;0;0.65;0;0.65;0;0.7;0;0.7;0;0.75;0;0.75;0;0.8;0;0.8;0;0.85;0;0.85;0;0.9;0;0.9;0;0.95;0;0.95;0;1;0.8;1;0.8#0;0;0.2;0.977778;1;1"
    build_gradient(pal)
    dic_windows[f'a_vol{i}'].setPalette('sulci-model')
    dic_windows[f'rvol{i}'] = a.fusionObjects(objects=[dic_windows[f'a_vol{i}']], method='VolumeRenderingFusionMethod')
    dic_windows[f'rvol{i}'].releaseAppRef()
    dic_windows[f'wvr{i}'] = a.createWindow('3D', block=block)
    dic_windows[f'wvr{i}'].addObjects(dic_windows[f'rvol{i}'])

# Visu distance map

In [5]:
# recompute
list_distance_map = []

#skel_dir = '/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/S.C.-sylv./mask/Lcrops'
skel_dir = '/neurospin/dico/data/deep_folding/current/datasets/UkBioBank40/crops/2mm/F.I.P./mask/Rcrops'
subjects_list = os.listdir(skel_dir)
subjects_list = [elem for elem in subjects_list if elem[-1]!='f']
subjects_list = subjects_list[:16]

for subject in subjects_list:
    skel_volume = aims.read(os.path.join(skel_dir, subject))
    skel_volume.np[skel_volume.np!=0]=1
    print(np.unique(skel_volume, return_counts=True))
    skel = skel_volume.np
    volume = skel[:,:,:,0]!=0
    structure = np.ones((3,3,3)) # we could also remove the edges
    #structure = None

    # Find connected components
    labeled_volume, num_components = find_connected_components(volume, structure=structure)
    print(f'Number of connected components : {num_components}')
    print(np.unique(labeled_volume, return_counts=True)[1])

    # Find minimax center for each connected component
    centers = {}
    for label_value in range(1, num_components + 1):  # Labels start from 1
        G = voxel_graph(labeled_volume, label_value)
        if G.number_of_nodes() > 0:
            center, max_dist, distances = find_minimax_center(G)
            centers[label_value] = (center, max_dist, distances)

    # fill distance_map with distances
    alpha=0.5
    distance_map = np.zeros(volume.shape)
    for label, (center, max_dist, distances) in centers.items():
        if max_dist==0: # avoid division by 0
            distance_map[center]=max_dist
        else:
            for coord, value in distances.items():
                distance_map[coord]=value*(1-alpha)/max_dist+alpha

    distance_map = np.expand_dims(distance_map, axis=-1).astype(np.float32)
    distance_vol = aims.Volume(distance_map)
    
    list_distance_map.append(distance_vol)

(array([0, 1], dtype=int16), array([75335,  1885]))
Number of connected components : 17
[75335  1427     3     6     2     1   207     6    12    92     8    31
     9    66    10     3     1     1]


In [12]:
block = a.createWindowsBlock(4)
dic_windows = {}

for i, vol in enumerate(list_distance_map):
    dic_windows[f'a_vol{i}'] = a.toAObject(vol)
    pal = a.createPalette('plasma')
    pal.header()['palette_gradients'] = f"0;0;0;0;0.05;0.4667;0.05;0.4667;0.1;0.5333;0.1;0.5333;0.15;0;0.15;0;0.2;0;0.2;0;0.25;0;0.25;0;0.3;0;0.3;0;0.35;0;0.35;0;0.4;0;0.4;0;0.45;0;0.45;0;0.5;0;0.5;0;0.55;0;0.55;0;0.6;0;0.6;0;0.65;0.7333;0.65;0.7333;0.7;0.9333;0.7;0.9333;0.75;1;0.75;1;0.8;1;0.8;1;0.85;1;0.85;1;0.9;0.8667;0.9;0.8667;0.95;0.8;0.95;0.8;1;0.8;1;0.8#0;0;0;0;0.05;0;0.05;0;0.1;0;0.1;0;0.15;0;0.15;0;0.2;0;0.2;0;0.25;0.4667;0.25;0.4667;0.3;0.6;0.3;0.6;0.35;0.6667;0.35;0.6667;0.4;0.6667;0.4;0.6667;0.45;0.6;0.45;0.6;0.5;0.7333;0.5;0.7333;0.55;0.8667;0.55;0.8667;0.6;1;0.6;1;0.65;1;0.65;1;0.7;0.9333;0.7;0.9333;0.75;0.8;0.75;0.8;0.8;0.6;0.8;0.6;0.85;0;0.85;0;0.9;0;0.9;0;0.95;0;0.95;0;1;0.8;1;0.8#0;0;0;0;0.05;0.5333;0.05;0.5333;0.1;0.6;0.1;0.6;0.15;0.6667;0.15;0.6667;0.2;0.8667;0.2;0.8667;0.25;0.8667;0.25;0.8667;0.3;0.8667;0.3;0.8667;0.35;0.6667;0.35;0.6667;0.4;0.5333;0.4;0.5333;0.45;0;0.45;0;0.5;0;0.5;0;0.55;0;0.55;0;0.6;0;0.6;0;0.65;0;0.65;0;0.7;0;0.7;0;0.75;0;0.75;0;0.8;0;0.8;0;0.85;0;0.85;0;0.9;0;0.9;0;0.95;0;0.95;0;1;0.8;1;0.8#0;0;0.2;0.977778;1;1"
    build_gradient(pal)
    dic_windows[f'a_vol{i}'].setPalette('plasma')
    dic_windows[f'rvol{i}'] = a.fusionObjects(objects=[dic_windows[f'a_vol{i}']], method='VolumeRenderingFusionMethod')
    dic_windows[f'rvol{i}'].releaseAppRef()
    dic_windows[f'wvr{i}'] = a.createWindow('3D', block=block)
    dic_windows[f'wvr{i}'].addObjects(dic_windows[f'rvol{i}'])